In [ ]:
from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import requests
import numpy as np
from pandas.io.json import json_normalize
import requests                  
from bs4 import BeautifulSoup    
import re                        
from datetime import datetime    
from time import mktime          

In [ ]:
##variables
url = "https://apidojo-yahoo-finance-v1.p.rapidapi.com/market/get-summary"
token = "22990f9b4amsh4df04e987b36e1bp18331cjsn862ed02f8065"
day_begin = "01-01-2010"
day_end = "01-01-2019"
interval = "1d"

In [ ]:
#START THE PROCESS
def start():
    Stockslist = list()
    Stocknames = list()
    End_of_Party = 0
    while End_of_Party == 0:
        input_stock = (input("Insert the Name of the Company to add stock, otherwise enter \"done\": "))
        input_stock = input_stock.upper()
        if input_stock == "DONE":
            End_of_Party += 1
        else:
            Stocknames.append(input_stock)
            input_stock = (input("Insert the SHORT NAME (Stockname) to add stock, otherwise enter \"done\": "))
            input_stock = input_stock.upper()
            if input_stock == "DONE":
                End_of_Party += 1
            else:
                Stockslist.append(input_stock)
            continue
        return Stocknames, Stockslist

In [ ]:
#activate the load data function
def get_data_rollin(Stocknames, Shortnames):
    index = 0
    oop = {}
    for names in Shortnames:
        print("We received a shit ton of data of",names)
        Stockname = Stocknames[index]
        index +=1
        oop[Stockname] = get_info(names)
    return oop
    
    
def get_info(Stock):
    Stock = load_csv_data(Stock, interval, day_begin, day_end)
    return Stock

In [ ]:
def _get_crumbs_and_cookies(stock):
    
    url = 'https://finance.yahoo.com/quote/{}/history'.format(stock)
    with requests.session():
        header = {'Connection': 'keep-alive',
                   'Expires': '-1',
                   'Upgrade-Insecure-Requests': '1',
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
                   AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
                   }
        
        website = requests.get(url, headers=header)
        soup = BeautifulSoup(website.text, 'lxml')
        crumb = re.findall('"CrumbStore":{"crumb":"(.+?)"}', str(soup))

        return (header, crumb[0], website.cookies)

In [ ]:
def convert_to_unix(date):
    
    datum = datetime.strptime(date, '%d-%m-%Y')
    
    return int(mktime(datum.timetuple()))


In [ ]:
def load_csv_data(stock, interval, day_begin, day_end):
  
    day_begin_unix = convert_to_unix(day_begin)
    day_end_unix = convert_to_unix(day_end)
    
    header, crumb, cookies = _get_crumbs_and_cookies(stock)
    
    with requests.session():
        url = 'https://query1.finance.yahoo.com/v7/finance/download/' \
              '{stock}?period1={day_begin}&period2={day_end}&interval={interval}&events=history&crumb={crumb}' \
              .format(stock=stock, day_begin=day_begin_unix, day_end=day_end_unix, interval=interval, crumb=crumb)
                
        website = requests.get(url, headers=header, cookies=cookies)
       
        return website.text.split('\n')[:-1]
    
    

In [ ]:
def next_step(List_of_raws):
    List_col = ["Date","Open","High","Low","Close","Adj Close","Volume"]
    Amazon = list()
    for data in List_of_raws:
        for col in List_col:
            data.split(",")
            print("We've just received a shit ton of data from",data)


In [ ]:
#change of column names except of Date:
x = {}
Col_dtype_to_float = ["Open","High","Low","Close","Adj Close","Volume"]
def changecolname(old_cols, Dataframe, NameofComp):
    for col in old_cols:
        x.update({col: NameofComp+col})
    Dataframe = Dataframe.rename(columns = x)
    return Dataframe
 
#df_alibaba = changecolname(Col_dtype_to_float, df_alibaba, "Alibaba")
#df_amazon = changecolname(Col_dtype_to_float, df_amazon, "Amazon")
#df_tencent = changecolname(Col_dtype_to_float, df_tencent, "Tencent")

In [ ]:
#def trigger_colnamechange(old_cols, Dataframe, NameofComp):

In [ ]:
def nene():
    empty = list()
    for x in Dataframe[Name + "Close"]:
        if len(empty) < 0.1:
            y = x
        perc = x / y
        empty.append(perc)
    Liste = empty
    return Liste

#ama = percentage(df_all, ama, "Amazon")
#ali = percentage(df_all, ali, "Alibaba")
#ten = percentage(df_all, ten, "Tencent")


In [100]:
def getting_data_seperated(List_of_raws):
    list2 = list()
    for x in List_of_raws.values():
        temp = list()
        for z in x:
            temp.append(z.split(","))            
        list2.append(temp)
    list2 = creating_dfs(Startprocess[0], list2)
    return list2
#df2 = pd.DataFrame(list2)
#df2.transpose()

In [ ]:
def change_col_name(temp, Colnames, Compname, compname):
    new_cols = list()
    compnameindex = 0 
    for name in temp[0]:
        new_cols.append(Compname)

In [ ]:

def creating_dfs(Compnames, List):
    df_all = pd.DataFrame(List[0])
    df_all = df_all.rename(columns=df_all.iloc[0])
    df_all = df_all.drop(0,axis=0)

    for i in range((len(Compnames)-1)):
        df = pd.DataFrame(List[i+1]) 
        df = df.rename(columns=df.iloc[0])
        df = df.drop(0,axis=0)
        df_all = pd.merge(df_all,df, on="Date") 

    return df_all

In [97]:

Colnames = ["Open","High","Low","Close","Adj Close","Volume"]
def renaiming_columns(df_all, column, companyname):
    index_dfall_col = 1
    index_comp_names = 0
    for comp in companyname:
        col_index = 0
        for col in column:
            if col_index < 8:
                df_all[index_dfall_col] = companyname[index_comp_names] + column[col_index]
                print(df_all[index_dfall_col])
                #print(df_all[index_dfall_col])
                col_index +=1
                index_dfall_col += 1
                continue
            else: continue
            index_dfall_col += 1
            col_index +=1
        index_comp_names += 1
    print(df_all)
    
    

In [ ]:
"""
Colnames = ["Open","High","Low","Close","Adj Close","Volume"]
def renaiming_columns(df_all, column, companyname):
    index_dfall_col = 1
    index_comp_names = 0 
    for x in companyname:
        for col in column:
            df_all[index_dfall_col] = companyname[index_comp_names] + col
            index_dfall_col += 1
        index_comp_names += 1
            
    print(df_all)
        
    

In [99]:
#renaiming_columns(df_all, Colnames, Startprocess[0])

In [ ]:
def calculating_percentages(df_all):

In [101]:
Startprocess = start()
List_of_raws = get_data_rollin(Startprocess[0],Startprocess[1])
#df = pd.DataFrame(List_of_raws)
df_all = getting_data_seperated(List_of_raws)
df_all

#SAP = SAP // APPLE = AAPL // AMAZON = AMZN // ADIDAS = ADDYY

Insert the Name of the Company to add stock, otherwise enter "done":  apple
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  aapl
Insert the Name of the Company to add stock, otherwise enter "done":  sap
Insert the SHORT NAME (Stockname) to add stock, otherwise enter "done":  sap
Insert the Name of the Company to add stock, otherwise enter "done":  done


We received a shit ton of data of AAPL
We received a shit ton of data of SAP
Date,Open,High,Low,Close,Adj Close,Volume
2009-12-31,30.447144,30.478571,30.080000,30.104286,19.996618,88102700
2010-01-04,30.490000,30.642857,30.340000,30.572857,20.307869,123432400
2010-01-05,30.657143,30.798571,30.464285,30.625713,20.342979,150476200
2010-01-06,30.625713,30.747143,30.107143,30.138571,20.019398,138040000
2010-01-07,30.250000,30.285715,29.864286,30.082857,19.982389,119282800
2010-01-08,30.042856,30.285715,29.865715,30.282858,20.115240,111902700
2010-01-11,30.400000,30.428572,29.778572,30.015715,19.937790,115557400
2010-01-12,29.884285,29.967142,29.488571,29.674286,19.710993,148614900
2010-01-13,29.695715,30.132856,29.157143,30.092857,19.989031,151473000
2010-01-14,30.015715,30.065714,29.860001,29.918571,19.873259,108223500
2010-01-15,30.132856,30.228571,29.410000,29.418571,19.541140,148516900
2010-01-19,29.761429,30.741428,29.605715,30.719999,20.405605,182501900
2010-01-20,30.701429,30.792856

,Date,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Adj Close_y,Volume_y
0,2009-12-31,30.447144,30.478571,30.080000,30.104286,19.996618,88102700,47.320000,47.450001,46.779999,46.810001,39.877140,588100
1,2010-01-04,30.490000,30.642857,30.340000,30.572857,20.307869,123432400,47.230000,47.630001,46.970001,47.110001,40.132710,1456800
2,2010-01-05,30.657143,30.798571,30.464285,30.625713,20.342979,150476200,46.980000,47.240002,46.630001,46.820000,39.885662,1565600
3,2010-01-06,30.625713,30.747143,30.107143,30.138571,20.019398,138040000,47.330002,48.009998,47.220001,47.910000,40.814224,1753300
4,2010-01-07,30.250000,30.285715,29.864286,30.082857,19.982389,119282800,48.639999,49.340000,48.540001,49.169998,41.887604,3681800
5,2010-01-08,30.042856,30.285715,29.865715,30.282858,20.115240,111902700,49.009998,49.750000,48.959999,49.540001,42.202808,1794900
6,2010-01-11,30.400000,30.428572,29.778572,30.015715,19.937790,115557400,50.400002,50.750000,50.080002,50.639999,43.139893,2293400
7,2010-01-12,29.884285,29.967142,29.488571,29.674286,19.710993,148614900,49.580002,49.939999,49.360001,49.549999,42.211327,1596500
8,2010-01-13,29.695715,30.132856,29.157143,30.092857,19.989031,151473000,50.599998,50.650002,50.099998,50.389999,42.926922,1973800
9,2010-01-14,30.015715,30.065714,29.860001,29.918571,19.873259,108223500,51.240002,51.250000,49.790001,50.160000,42.730984,8310000


In [ ]:
#df2 = pd.DataFrame(list2[0])
#df3 = pd.DataFrame(list2[1])